In [33]:
import pandas as pd
import spacy
from itertools import chain
import json

In [13]:
PIPELINE = "en_core_web_md"
nlp = spacy.load(PIPELINE)
nlp.add_pipe("sentencizer")

In [5]:
df = pd.read_csv("../data/wikidata/events_wikidata.csv")


In [14]:
events = df["event_text"].tolist()

In [51]:
first_sentences= df["first_sentence"].tolist()

In [15]:
sents=[]
for event_text in events:
    doc = nlp(event_text)
    sentences= [sent.text.strip() for sent in doc.sents]
    sents.append(sentences)

In [18]:
sents_all = list(chain.from_iterable(sents))

In [19]:
sents_all[0]

'Russian troops enter the Chernobyl Exclusion Zone.'

In [21]:
verb_list = []
verb_pos_list = []
noun_list = []
noun_pos_list = []

In [39]:
def get_verbs_nouns(doc):
    for word in doc:
        if word.dep_ == "ROOT":
            for subword in word.children: 
                if subword.dep_ == "dobj": 
                    verb_list.append(word.lemma_)
                    verb_pos_list.append(word.pos_)
                    noun_list.append(subword.lemma_)
                    noun_pos_list.append(subword.pos_)
    

In [ ]:
for sent in sents_all:
    if "that" in sent:
        print(sent)
        vortext = sent.split("that")[0]
        subtext = sent.split("that")[1]
        doc = nlp(subtext)
        get_verbs_nouns(doc)
    else:
        doc=nlp(sent)
        get_verbs_nouns(doc)

In [32]:
verb_list = list(set(verb_list))

In [34]:
with open("../data/events/verbs.json", "w") as f:
    json.dump(verb_list, f)

In [37]:
nouns = ['curfew', 'wipeout', 'control', 'relocation', 'convoy', 
         'control', 'war', 'assault', 'attack', 'offensive', 'protest',
         'operation', 'ban', 'request', 'spread']

In [48]:
def actor_extractor(root): 
    for child in root.children: 
        if child.dep_ == "nsubj": 
            nsubj = child.text 
            nsubj_subtree = ''.join(w.text_with_ws for w in child.subtree).strip() 
            return nsubj_subtree 

In [41]:
with open("../data/events/verbs.json",) as f:
    verb_list = json.load(f)

In [55]:
def detect_event(doc): 
    for word in doc: 
        if word.dep_ == "ROOT": 
            # for subword in word.children: 
            #     if subword.dep_ == "dobj" and subword.lemma_ in dobj_list: 
            return word  

In [63]:
actors = get_actors(first_sentences)

Russian troops enter the Chernobyl Exclusion Zone.
actor: Russian troops
********************
 russian navy warships have attacked snake island near the danube delta.
actor: russian navy warships
Russian troops enter Sumy and take control of the road leading from Kyiv to Moscow.
actor: Russian troops
********************
Russian gunships bombard Hostomel Airport near Kyiv.
actor: Russian gunships
********************
An-225 Mriya, the largest aircraft in the world by weight and by wingspan, is destroyed during a Russian air strike on the suburb of Kyiv.
actor: None
********************
The Mayor of Kyiv, Vitali Klitschko, announces a curfew in the capital from 10 pm to 7 am.
actor: The Mayor of Kyiv, Vitali Klitschko,
********************
Russian President Vladimir Putin orders a military operation to "demilitarise and denazify" Ukraine.
actor: Russian President Vladimir Putin
********************
 russian ground forces have entered luhansk, sumy, kharkiv, kherson, chernihiv, and zhyto

In [66]:
df["first_actor"] =actors

In [67]:
df.to_csv("../data/events/events_actors.csv", index=False)

In [62]:
def get_actors(sents_all):
    actors=[]
    for sent in sents_all:
        if "that" in sent:
            sent = sent.capitalize()
            vortext = sent.split("that")[0]
            subtext = sent.split("that")[1]
            print(subtext)
            doc = nlp(subtext)
            root = detect_event(doc)
            if root:
                actor = actor_extractor(root)

                print("actor:", actor)
                if actor in ["they", "it", "he", "she"]:
                    print(vortext)
                    doc_ = nlp(vortext)
                    root_ = detect_event(doc_)
                    if root_:
                        actor_ = actor_extractor(root_)

                        print("actor:", actor_)
                        actors.append(actor_)
                else:
                    actors.append(actor)
            else:
                actors.append(None)
                    
                    
                print("*"*20)
                # get_verbs_nouns(doc)
        else:
            doc=nlp(sent)
            root = detect_event(doc)
            if root:
                actor = actor_extractor(root)

                print(sent)
                print("actor:", actor)
                print("*"*20)
                actors.append(actor)
            else:
                actors.append(None)
            # get_verbs_nouns(doc)
    return actors

In [7]:
texts = df["first_sentence"].tolist()

In [ ]:

for text in texts:
    if "that" in text:
        vortext = text.split("that")[0]
        subtext = text.split("that")[1]
        print(vortext)
        print(subtext)
        print("="*20)